In [1]:
movs = '/mnt/d/mystuff/pwgis/mod5/mov/movrst.tif'

_var = '/mnt/d/mystuff/pwgis/mod5/var/'

refrst = '/mnt/d/mystuff/pwgis/mod5/mdt.tif'

out = '/mnt/d/mystuff/pwgis/mod5/ivmad.tif'

In [2]:
import os
import math as m
import datetime as dt
from glass.gp.ext   import rstext_to_rst
from glass.prop     import check_isRaster
from glass.prop.rst import rst_shape, frequencies
from glass.wenv.grs import run_grass
from glass.pys.oss    import lst_ff, fprop

In [3]:
time_a = dt.datetime.now().replace(microsecond=0)

In [4]:
# Get reference raster
ws = os.path.dirname(out)

refrst = rstext_to_rst(refrst, os.path.join(ws, 'refrst.tif'))

In [5]:
# List raster files

rstvar = lst_ff(_var, file_format='tif')

# Get Reference shape
refshape = rst_shape(refrst)

# Get Variables Raster Shape and see if there is any difference
varshp = rst_shape(rstvar)

for r in varshp:
    if varshp[r] != refshape:
        raise ValueError((
            f'All rasters must have the same dimension! '
            f'{r} have different shape when compared with refrst!'
        ))

In [6]:
rstvar

['/mnt/d/mystuff/pwgis/mod5/var/aspect.tif',
 '/mnt/d/mystuff/pwgis/mod5/var/elevation.tif',
 '/mnt/d/mystuff/pwgis/mod5/var/geologia.tif',
 '/mnt/d/mystuff/pwgis/mod5/var/slope.tif',
 '/mnt/d/mystuff/pwgis/mod5/var/varcurv.tif',
 '/mnt/d/mystuff/pwgis/mod5/var/varinvtopoidx.tif',
 '/mnt/d/mystuff/pwgis/mod5/var/varroads.tif',
 '/mnt/d/mystuff/pwgis/mod5/var/vartiposolo.tif',
 '/mnt/d/mystuff/pwgis/mod5/var/varusosolo2.tif',
 '/mnt/d/mystuff/pwgis/mod5/var/varwater.tif']

In [7]:
# Start GRASS GIS Session

# Get name for GRASS GIS location
loc = fprop(movs, 'fn', forceLower=True)[:5] + '_loc'

# Create GRASS GIS location
gbase = run_grass(ws, location=loc, srs=refrst)

# Start GRASS GIS Session
import grass.script.setup as gsetup

gsetup.init(gbase, ws, loc, 'PERMANENT')

'/tmp/tmpk6rkzhpn'

In [8]:
from glass.it.shp  import shp_to_grs
from glass.it.rst   import rst_to_grs, grs_to_rst
from glass.dp.torst import grsshp_to_grsrst
from glass.rst.rcls import category_rules, rcls_rst
from glass.rst.alg import rstcalc

In [9]:
# Check if movs are raster
is_rst = check_isRaster(movs)

if is_rst:
    movrst = rst_to_grs(movs, fprop(movs, 'fn', forceLower=True))

else:
    movshp = shp_to_grs(movs, fprop(movs, 'fn', forceLower=True), asCMD=True)
    
    # To raster
    movrst = grsshp_to_grsrst(movshp, 1, f'rst_{movshp}', cmd=True)

In [10]:
# Add rasters to GRASS GIS
grsvar = [rst_to_grs(r, fprop(r, 'fn', forceLower=True)) for r in rstvar]

In [11]:
# Get raster representing areas with values
gref = rst_to_grs(refrst, 'refrst')
grs_to_rst(gref, os.path.join(ws, loc, f"{gref}.tif"), as_cmd=True)

i = 1
for r in grsvar:
    gref = rstcalc(f"int({r} * {gref})", f"refrst_{str(i)}", api='grass')
    grs_to_rst(gref, os.path.join(ws, loc, f"{gref}.tif"), as_cmd=True)
    i += 1

In [12]:
# Ensure that we have only cells with data in all rasters

refrules = category_rules({0 : 1}, os.path.join(ws, loc, 'refrules.txt'))

gref = rcls_rst(gref, refrules, f'rcls_{gref}', api="pygrass")
#grs_to_rst(gref, os.path.join(ws, loc, f"{gref}.tif"), as_cmd=True)

grsvar = [rstcalc(f"{r} * {gref}", f"{r}_san", api='grass') for r in grsvar]

In [13]:
# Export rasters to get frequencies
filevar = {r : grs_to_rst(r, os.path.join(
    ws, loc, f"{r}.tif"
), as_cmd=True) for r in grsvar}

In [14]:
# Get raster frequencies

# Negative nodata values are not allowed
rstfreq = {r : frequencies(filevar[r]) for r in filevar}

In [15]:
# Count total cells
i = 0
for r in rstfreq:
    if not i:
        totalcells = 0
        for v in rstfreq[r]:
            totalcells += rstfreq[r][v]
        
        i += 1
    
    else:
        __totalcells = 0
        for v in rstfreq[r]:
            __totalcells += rstfreq[r][v]
        
        if __totalcells != totalcells:
            raise ValueError(f'{r} has a different number of cells with data')

In [16]:
# Intersect landslides raster with var rasters
varwithmov = [rstcalc(f"{movrst} * {r}", f"mov_{r}", api='grass') for r in grsvar]

In [17]:
# Export rasters to get frequencies
filemov = {grsvar[i] : grs_to_rst(varwithmov[i], os.path.join(
    ws, loc, f"{varwithmov[i]}.tif"
), as_cmd=True) for i in range(len(varwithmov))}

In [18]:
# Get raster frequencies

# Negative nodata values are not allowed
rstmovfreq = {r : frequencies(filemov[r]) for r in filemov}

In [19]:
# Count total cells with landslides
i = 0
for r in rstmovfreq:
    if not i:
        totalmov = 0
        for v in rstmovfreq[r]:
            totalmov += rstmovfreq[r][v]
        
        i += 1
    
    else:
        __totalmov = 0
        for v in rstmovfreq[r]:
            __totalmov += rstmovfreq[r][v]
        
        if __totalmov != totalmov:
            raise ValueError(f'{r} has a different number of cells with data')

In [20]:
# Estimate VI for each class of every variable

vi = {}

denom = totalmov / totalcells
for r in rstfreq:
    vi[r] = {}
    for cls in rstfreq[r]:
        if cls in rstmovfreq[r]:
            vi[r][cls] = m.log10(
                (rstmovfreq[r][cls] / rstfreq[r][cls]) / denom
            )
        
        else:
            vi[r][cls] = 9999

In [21]:
# Replace Classes without VI, from 9999 to minimum VI
vis = []
for d in vi.values():
    vis += d.values()

min_vi = int(round(min(vis), 4) * 10000)

for r in vi:
    for cls in vi[r]:
        if vi[r][cls] == 9999:
            vi[r][cls] = min_vi
        else:
            vi[r][cls] = int(round(vi[r][cls], 4) * 10000)

In [22]:
# Reclassify

vivar = []
for r in grsvar:
    rules = category_rules(
        vi[r], os.path.join(ws, loc, f'vi_{r}.txt')
    )

    virst = rcls_rst(r, rules, f'vi_{r}', api="pygrass")
    
    vivar.append(virst)

# Integer to float
virst = [rstcalc(f"{r} / 10000.0", f"{r}_f", api='grass') for r in vivar]

In [23]:
# Sum results
virstfinal = rstcalc(" + ".join(virst), fprop(out, 'fn'), api='grass')

fffinal = grs_to_rst(virstfinal, out)

In [24]:
time_b = dt.datetime.now().replace(microsecond=0)

In [25]:
print(time_b - time_a)

0:01:56
